# Training car detection model using YOLOv8

In [1]:
import os.path
import json
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import shutil
import random
from constants import LABEL_MAPPING

### Utility to copy files

In [3]:
k = 670

for i in range(0, 15):
    for j in range(0, 10):
        shutil.copy(f'../../data/recordings/cross_yellow/{i}/videos/{j}.png', f'../../data/recordings/images/{k}.png')
        k += 1

In [2]:
def view_features(img_path, xtl, ytl, xbr, ybr, points):
    img = cv2.imread(img_path)
    w = xbr - xtl
    h = ybr - ytl

    cv2.rectangle(img, (xtl, ytl), (xbr, ybr), [255,0,0], 1)
    cv2.circle(img,(w, h), 1, [255,0,0], -1)

    visible_color = [0,255,0]
    invisible_color = [0,0, 255]
    for p in points:
        x, y, visible = p 
        cv2.circle(img, (x, y), 1, visible_color if visible else invisible_color, -1)

    cv2.imshow('Image', img) 
    cv2.waitKey(0) 
    cv2.destroyAllWindows() 

### Extract features from video annotations

In [4]:
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080

# Opening JSON file
f = open('../../data/cars/annotations.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)

out_dir = '../../data/cars/labels/all'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

temp = 0
for item in data['items']:
    label_file = open(os.path.join(out_dir, item['id'] + '.txt'), 'w')

    sorted_annotations = sorted(item['annotations'], key=lambda x: x['label_id'])

    xtl = int(sorted_annotations[0]['bbox'][0])
    ytl = int(sorted_annotations[0]['bbox'][1])
    w = int(sorted_annotations[0]['bbox'][2])
    h = int(sorted_annotations[0]['bbox'][3])
    xbr = xtl + w 
    ybr = ytl + h

    label_file.write('0 {} {} {} {} '.format(str((xtl + (w / 2)) / IMAGE_WIDTH), str((ytl + (h / 2)) / IMAGE_HEIGHT),
                                            str(w / IMAGE_WIDTH), str(h / IMAGE_HEIGHT)))
    points = []
    for i in range(1, 17):
        px = int(sorted_annotations[i]['points'][0])
        py = int(sorted_annotations[i]['points'][1])
        visible = 0 if sorted_annotations[i]['attributes']['occluded'] == True else 2
        # Switch append to view a specific feature using LABEL_MAPPING
        points.append([px, py, visible])
        #points.append([int(sorted_annotations[LABEL_MAPPING['front_right_tire_back']]['points'][0]), int(sorted_annotations[LABEL_MAPPING['front_right_tire_back']]['points'][1]), 2])

    for p_, (px, py, visible) in enumerate(points):
        label_file.write('{} {} {}'.format(px / IMAGE_WIDTH, py / IMAGE_HEIGHT, visible))
        if p_ < len(points) - 1:
            label_file.write(' ')
        else:
            label_file.write('\n')

    img_path = '../../data/cars/images/all/%s.png' % item['id']

    if temp == 10:
        view_features(img_path, xtl, ytl, xbr, ybr, points)

    label_file.close()
    temp += 1

# Closing file
f.close()

### Fill in missing labels for empty images

In [20]:
def fill_missing_labels(directory, start):
    existing_numbers = set()

    files = os.listdir(directory)
    end = len(files) - 1
    for filename in os.listdir(directory):
        try:
            file_number = int(filename.split('.')[0])
            existing_numbers.add(file_number)
        except ValueError:
            pass  # Ignore files that do not match the expected naming pattern

    missing_numbers = set(range(start, end + 1)) - existing_numbers
    return sorted(missing_numbers)

In [21]:
labels = fill_missing_labels('../../data/cars/labels/all', 0)
print(labels)

for number in labels:
    label_file = open('../../data/cars/labels/all/%s.txt' % number, 'w')
    label_file.write('0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0')
    label_file.close()

[]


### Split validation data

In [17]:
train_label_folder = '../../data/cars/images/train'
train_image_folder = '../../data/cars/labels/train'
val_label_folder = '../../data/cars/images/val'
val_image_folder = '../../data/cars/labels/val'
shutil.rmtree(train_label_folder, ignore_errors=True)
shutil.rmtree(train_image_folder, ignore_errors=True)
shutil.rmtree(val_label_folder, ignore_errors=True)
shutil.rmtree(val_image_folder, ignore_errors=True)

label_files = os.listdir('../../data/cars/labels/all')
image_files = os.listdir('../../data/cars/images/all')

assert(len(label_files) == len(image_files))

os.makedirs(train_label_folder)
os.makedirs(train_image_folder)
os.makedirs(val_label_folder)
os.makedirs(val_image_folder)

random.shuffle(image_files)
split_index = int(0.8 * len(image_files))

train_files = []
for i in range(0, split_index):
    train_files.append(os.path.basename(image_files[i]).split('.')[0])

for file in train_files:
    shutil.copy(f'../../data/cars/images/all/{file}.png', f'../../data/cars/images/train')
    shutil.copy(f'../../data/cars/labels/all/{file}.txt', f'../../data/cars/labels/train')

val_files = []
for i in range(split_index, len(image_files)):
    val_files.append(os.path.basename(image_files[i]).split('.')[0])

for file in val_files:
    shutil.copy(f'../../data/cars/images/all/{file}.png', f'../../data/cars/images/val')
    shutil.copy(f'../../data/cars/labels/all/{file}.txt', f'../../data/cars/labels/val')